In [1]:
import pandas as pd
import numpy as np

# Exploration

In [2]:
airbnb = pd.read_csv("airbnb_train.csv")
airbnb.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds
0,5708593,4.317488,House,Private room,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",3,1.0,Real Bed,flexible,False,...,NaN,33.782712,-118.134410,Island style Spa Studio,Long Beach,0,NaN,90804,0.0,2.0
1,14483613,4.007333,House,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",4,2.0,Real Bed,strict,False,...,2017-09-17,40.705468,-73.909439,"Beautiful and Simple Room W/2 Beds, 25 Mins to...",Ridgewood,38,86.0,11385,1.0,2.0
2,10412649,7.090077,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",6,2.0,Real Bed,flexible,False,...,NaN,38.917537,-77.031651,2br/2ba luxury condo perfect for infant / toddler,U Street Corridor,0,NaN,20009,2.0,2.0
3,17954362,3.555348,House,Private room,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1,1.0,Real Bed,flexible,True,...,2017-09-29,40.736001,-73.924248,Manhattan view from Queens. Lovely single room .,Sunnyside,19,96.0,11104,1.0,1.0
4,9969781,5.480639,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,moderate,True,...,2017-08-28,37.744896,-122.430665,Zen Captured Noe Valley House,Noe Valley,15,96.0,94131,2.0,2.0


# Entraînement

### Remplacer le type de propriété par un indice, cela permet à l’algo de l’utiliser

In [3]:
class CustomTransformation():

    def __init__(self):
        """
        Class simple pour convertir les type de propriétés en des indices numériques, utilisable pour un algo de machine learning
        """

        self.fitted = False # Indique si fit_transform a été utilisé, pour éviter d’utiliser transform sans que fit ai été appelé
        self.property2index = dict() # Dictionnaire qui va convertir le nom en indice
        self.max_index = 0 # Indique le dernier indice de la propriété.

    def fit_transform(self, dataset):

        self.fitted = True

        # Récupère les types de propriété (maison, appart etc...)
        properties = dataset["property_type"].unique()
        print("Tous les types de propriétés : ", properties)

        self.property2index = {prop:i for (i, prop) in enumerate(properties)}
        self.max_index = max(list(self.property2index.values()))

        # transform
        return self.transform(dataset)
    
    def transform(self, dataset):
        # Transform les propriétés en indice
        dataset.loc[:, "property_type"] = dataset["property_type"].replace(self.property2index)


        # Ligne un peu moche qui fait en sorte de remplacer les lignes qui ont des nom de logement qui n’était pas dans l’entrainement
        dataset.loc[dataset["property_type"].map(type).eq(str), "property_type"] = np.nan

        
        # remplace les valeurs null
        dataset[dataset.bathrooms.isna()] = 0
        dataset[dataset.accommodates.isna()] = 0
        dataset[dataset.property_type.isna()] = self.max_index + 1
        return dataset

In [4]:
features_transformer = CustomTransformation()

airbnb.head()
airbnb_train = features_transformer.fit_transform(airbnb)
airbnb_train.head()

Tous les types de propriétés :  ['House' 'Apartment' 'Townhouse' 'Guest suite' 'Condominium' 'Timeshare'
 'Chalet' 'Guesthouse' 'Bungalow' 'Loft' 'In-law' 'Boat' 'Dorm' 'Other'
 'Bed & Breakfast' 'Camper/RV' 'Villa' 'Boutique hotel' 'Cabin' 'Hostel'
 'Hut' 'Yurt' 'Serviced apartment' 'Castle' 'Vacation home' 'Tent' 'Cave'
 'Tipi' 'Earth House' 'Island' 'Treehouse']


,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,zipcode,bedrooms,beds
0,5708593,4.317488,0,Private room,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",3,1.0,Real Bed,flexible,False,...,NaN,33.782712,-118.134410,Island style Spa Studio,Long Beach,0,NaN,90804,0.0,2.0
1,14483613,4.007333,0,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",4,2.0,Real Bed,strict,False,...,2017-09-17,40.705468,-73.909439,"Beautiful and Simple Room W/2 Beds, 25 Mins to...",Ridgewood,38,86.0,11385,1.0,2.0
2,10412649,7.090077,1,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",6,2.0,Real Bed,flexible,False,...,NaN,38.917537,-77.031651,2br/2ba luxury condo perfect for infant / toddler,U Street Corridor,0,NaN,20009,2.0,2.0
3,17954362,3.555348,0,Private room,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1,1.0,Real Bed,flexible,True,...,2017-09-29,40.736001,-73.924248,Manhattan view from Queens. Lovely single room .,Sunnyside,19,96.0,11104,1.0,1.0
4,9969781,5.480639,0,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,moderate,True,...,2017-08-28,37.744896,-122.430665,Zen Captured Noe Valley House,Noe Valley,15,96.0,94131,2.0,2.0


In [5]:
class FeatureSelection():

    def __init__(self):
        """
        Class simple pour juste garder les colonnes qui nous intéresse
        N'a pas forcément l'air nécessaire, mais c'est pour être sur que j'applique bien le même process au train et au test
        """

    def fit_transform(self, dataset, y=None):
        return self.transform(dataset)
    
    def transform(self, dataset):
        new_dataset = dataset[["property_type", "accommodates", "bathrooms"]].copy()

        for col in new_dataset.columns:
            new_dataset[col] = pd.to_numeric(new_dataset[col]) # Converti tout en nombre
        return new_dataset
    
feature_selector = FeatureSelection()

airbnb_train = feature_selector.transform(airbnb)
airbnb_train.head()

,property_type,accommodates,bathrooms
0,0,3,1.0
1,0,4,2.0
2,1,6,2.0
3,0,1,1.0
4,0,4,1.0


### Apprentissage

In [6]:
from sklearn.model_selection import train_test_split

# Vous avez le droit d’utiliser les Pipeline et transform de sklearn : 
# https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
features_transformer = CustomTransformation()
feature_selector = FeatureSelection()

airbnb = pd.read_csv("airbnb_train.csv")

airbnb_train = features_transformer.fit_transform(airbnb)
airbnb_train = feature_selector.transform(airbnb_train)

X = airbnb_train.copy()
y = airbnb["log_price"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

Tous les types de propriétés :  ['House' 'Apartment' 'Townhouse' 'Guest suite' 'Condominium' 'Timeshare'
 'Chalet' 'Guesthouse' 'Bungalow' 'Loft' 'In-law' 'Boat' 'Dorm' 'Other'
 'Bed & Breakfast' 'Camper/RV' 'Villa' 'Boutique hotel' 'Cabin' 'Hostel'
 'Hut' 'Yurt' 'Serviced apartment' 'Castle' 'Vacation home' 'Tent' 'Cave'
 'Tipi' 'Earth House' 'Island' 'Treehouse']


Le score R2 est un score de regression, il vaut 1 si la prédiction est parfaite, 0 si la valeur prédite est la moyenne de $y$. Et des scores négatifs si la prédiction est moins bonne que prédire la moyenne (donc vraiment mauvais)

In [7]:
from sklearn.svm import LinearSVR
from sklearn.metrics import r2_score

model = LinearSVR()

model.fit(X_train, y_train)

y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)


print(f"Score en entrainenement : {r2_score(y_true=y_train, y_pred=y_pred_train)}")
print(f"Score en test : {r2_score(y_true=y_test, y_pred=y_pred_test)}")


C:\Users\Mseurin\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


Score en entrainenement : 0.3232143415232205
Score en test : 0.3134678837065975


C:\Users\Mseurin\AppData\Roaming\Python\Python38\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


## Prédiction sur le fichier de test

In [8]:
airbnb_test = pd.read_csv("airbnb_test.csv")

# J’applique le même traitement que mon fichier entraînement
final_X_test = features_transformer.transform(airbnb_test)
final_X_test = feature_selector.transform(final_X_test)

final_X_test.tail()

,property_type,accommodates,bathrooms
51872,0,4,1.0
51873,0,4,1.0
51874,1,3,1.0
51875,0,4,1.0
51876,1,3,1.0


In [9]:
y_final_prediction = model.predict(final_X_test)
print(y_final_prediction)

[4.68973885 5.44945555 5.0695972  ... 4.68973885 4.86993483 4.68973885]


## Sauvegarde dans le fichier de prédiction

In [10]:
prediction_example = pd.read_csv("prediction_example.csv")
prediction_example["logpred"] = y_final_prediction

prediction_example.to_csv("MaPredictionFinale.csv", index=False) # index=False pour éviter d’ajouter l’index interne à pandas
# Voilà !

## Test de votre fichier

In [13]:
def estConforme(monFichier_csv):
    votre_prediction = pd.read_csv(monFichier_csv)

    fichier_exemple = pd.read_csv("prediction_example.csv")

    assert votre_prediction.columns[1] == fichier_exemple.columns[1], f"Attention, votre colonne de prédiction doit s'appeler {fichier_exemple.columns[1]}, elle s'appelle '{votre_prediction.columns[1]}'"
    assert len(votre_prediction) == len(fichier_exemple), f"Attention, vous devriez avoir {len(fichier_exemple)} prédiction dans votre fichier, il en contient '{len(votre_prediction)}'"

    assert np.all(votre_prediction.iloc[:,0] == fichier_exemple.iloc[:, 0])

    print("Fichier conforme!")

estConforme("MaPredictionFinale.csv")

Fichier conforme!


# Ce que je vais faire de mon côté

In [12]:
# Vous n’avez pas accès à ce fichier, c’est normal, ce sont les vrais prédictions
# ===============================================================================
# true_test = pd.read_csv("../true_prediction.csv") 
# ==========================================================

# votre_prediction = pd.read_csv("MaPredictionFinale.csv")["logpred"]
# print(r2_score(y_pred=votre_prediction, y_true=true_test["log_price"]))

# votre_prediction = pd.read_csv("prediction_example.csv")["logpred"] # Devrait avoir ~0
# print(r2_score(y_pred=votre_prediction, y_true=true_test["log_price"]))